In [1]:
import os
import sys

import django

sys.path.append('../')  # add path to project root dir

os.environ["DJANGO_SETTINGS_MODULE"] = "flourish.settings"

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

django.setup()

  * Reading config from flourish.ini
Loading Data Encryption (init)...
 * loading keys from /Users/nimrodmunatsi/source/flourish-code-space/flourish/crypto_fields
 * loading rsa.restricted.public ... Done.
 * loading rsa.restricted.private ... Done.
 * loading rsa.local.public ... Done.
 * loading rsa.local.private ... Done.
 * loading aes.local ... Done.
 * loading aes.restricted ... Done.
 * loading salt.local ... Done.
 * loading salt.restricted ... Done.
 Done loading Data Encryption (init)...


/Users/nimrodmunatsi/.venv/flourish/lib/python3.9/site-packages/django_q/conf.py:139: UserWarning: Retry and timeout are misconfigured. Set retry larger than timeout, 
        failure to do so will cause the tasks to be retriggered before completion. 
        See https://django-q.readthedocs.io/en/latest/configure.html#retry for details.
  warn(
/Users/nimrodmunatsi/.venv/flourish/lib/python3.9/site-packages/simple_history/models.py:164: UserWarning: HistoricalRecords added to abstract model (HistoryManagerMixin) without inherit=True
  warnings.warn(msg, UserWarning)
/Users/nimrodmunatsi/.venv/flourish/lib/python3.9/site-packages/simple_history/models.py:164: UserWarning: HistoricalRecords added to abstract model (ScheduleModelMixin) without inherit=True
  warnings.warn(msg, UserWarning)
/Users/nimrodmunatsi/.venv/flourish/lib/python3.9/site-packages/simple_history/models.py:164: UserWarning: HistoricalRecords added to abstract model (VisitModelMixin) without inherit=True
  warnings.wa

Loading Data Encryption ...
 * found encryption keys in /Users/nimrodmunatsi/source/flourish-code-space/flourish/crypto_fields.
 * using model django_crypto_fields.crypt.
 Done loading Data Encryption.
Loading Edc Consent ...
 * checking for site consents ...
 * registered consents 'consents' from 'flourish_caregiver'
 * registered consents 'consents' from 'flourish_child'
 * registered consents 'consents' from 'pre_flourish'
 * registered consents 'consents' from 'flourish_facet'
 * flourish_caregiver.subjectconsent 1 covering 2020-07-01 UTC to 2025-06-30 UTC
 * flourish_caregiver.subjectconsent 2 covering 2020-07-01 UTC to 2025-06-30 UTC
 * flourish_caregiver.subjectconsent 3 covering 2020-07-01 UTC to 2025-06-30 UTC
 * flourish_caregiver.subjectconsent 4 covering 2020-07-01 UTC to 2025-06-30 UTC
 * flourish_caregiver.tbadolconsent 1 covering 2020-07-01 UTC to 2025-06-30 UTC
 * flourish_caregiver.tbinformedconsent 1 covering 2020-07-01 UTC to 2025-06-30 UTC
 * flourish_child.childdum

In [2]:
from django.db.models import Count
from flourish_caregiver.models import SubjectConsent


def mothers_with_multiple_kids():
    consents = SubjectConsent.objects.annotate(
        child_count=Count('caregiverchildconsent')
    ).filter(child_count__gt=1)
    return consents.values_list('subject_identifier', flat=True)

In [3]:

from itertools import combinations


def find_objects_within_90_days(object_list):
    """
    Given a list of SubjectConsent objects, 
    return the first pair of objects found that have date difference within 90 days.
    If no such pair is found, return None.
    """
    for obj1, obj2 in combinations(object_list, 2):
        if abs((obj1.report_datetime - obj2.report_datetime).days) <= 90:
            return obj1, obj2
    return None

In [4]:
def compare_instances(instance1, instance2, exclude_fields=None):
    """
    Compare two Django model instances field by field.

    Parameters:
        instance1, instance2: The instances to compare.
        exclude_fields: A list of field names to exclude from comparison. Defaults to None.

    Returns:
        dict: A dictionary of field names and boolean values indicating whether the 
              values for that field in instance1 and instance2 are equal.
    """
    assert instance1.__class__ == instance2.__class__, "Instances must be of the same class."

    comparison_dict = {}

    field_names = [f.name for f in instance1._meta.fields]
    if exclude_fields:
        field_names = [f for f in field_names if f not in exclude_fields]

    for field in field_names:
        comparison_dict[field] = (getattr(instance1, field) == getattr(instance2, field))

    return comparison_dict


In [5]:
from tqdm import tqdm
from flourish_caregiver.models import MaternalVisit
from django.apps import apps as django_apps

mothers_with_different_data = {}

for mother in tqdm(mothers_with_multiple_kids()):
    unique_mother_visits_codes = list(
        set(MaternalVisit.objects.filter(subject_identifier=mother).values_list(
            'visit_code', flat=True)))
    for unique_visit_code in unique_mother_visits_codes:
        unique_visit = MaternalVisit.objects.filter(subject_identifier=mother,
                                                    visit_code=unique_visit_code).earliest(
            'report_datetime')
        if getattr(unique_visit, 'visits', None) is not None:
            visit_crfs = unique_visit.visits.get(unique_visit.visit_code).crfs
            for visit_crf in visit_crfs:
                crf_model_cls = django_apps.get_model(visit_crf.model)
                crf_model_objs = crf_model_cls.objects.filter(
                    maternal_visit__subject_identifier=mother,
                    maternal_visit__visit_code=unique_visit.visit_code)
                exclude_fields = ['id', '_state', 'created', 'modified', 'user_created','device_created','device_modified','form_as_json', 'site','consent_model','consent_version','maternal_visit','report_datetime','user_modified','hostname_created','hostname_modified','revision']
                if crf_model_objs.count() > 1:
                    obj_pair = find_objects_within_90_days(crf_model_objs)
                    if obj_pair:
                        obj1, obj2 = obj_pair
                        comparison_result = compare_instances(obj1, obj2, exclude_fields)
                        for field, equal in comparison_result.items():
                            if not equal:
                                if mother not in mothers_with_different_data:
                                    mothers_with_different_data[mother] = []
                                if visit_crf.model not in mothers_with_different_data[
                                    mother]:
                                    mothers_with_different_data[mother].append(
                                        f'{visit_crf.model}_{unique_visit_code}')
                                break
mothers_with_different_data

100%|██████████| 128/128 [00:23<00:00,  5.49it/s]


{'B142-040991009-7': ['flourish_caregiver.medicalhistory_2004M',
  'flourish_caregiver.relationshipfatherinvolvement_2004M',
  'flourish_caregiver.medicalhistory_2005M',
  'flourish_caregiver.medicalhistory_2004M',
  'flourish_caregiver.relationshipfatherinvolvement_2004M',
  'flourish_caregiver.medicalhistory_2005M'],
 'B142-040990804-2': ['flourish_caregiver.sociodemographicdata_2002M',
  'flourish_caregiver.covid19_2002M',
  'flourish_caregiver.sociodemographicdata_2002M',
  'flourish_caregiver.covid19_2002M'],
 'B142-040990837-2': ['flourish_caregiver.sociodemographicdata_2008M',
  'flourish_caregiver.sociodemographicdata_2006M',
  'flourish_caregiver.sociodemographicdata_2007M'],
 'B142-040990639-2': ['flourish_caregiver.breastfeedingquestionnaire_2002M',
  'flourish_caregiver.breastfeedingquestionnaire_2002M',
  'flourish_caregiver.breastfeedingquestionnaire_2002M'],
 'B142-040990096-5': ['flourish_caregiver.sociodemographicdata_3000M',
  'flourish_caregiver.medicalhistory_3000M'